In [53]:
import xgboost as xgb
import numpy as np
import pandas as pd
import time
import pickle
import os
from math import log
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, ParameterGrid,StratifiedKFold, cross_val_score

from sklearn import metrics 
from sklearn.metrics import confusion_matrix, precision_recall_curve, average_precision_score, brier_score_loss, f1_score, log_loss,auc,roc_curve,auc

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.decomposition import PCA
#from sklearn.utils.fixes import signature

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [54]:
import pandas as pd
import numpy as np

In [55]:
pbdf = pd.read_csv("..\\PBData\\INCEPTION_MATCH_FINAL2.csv")
mesdq_df = pd.read_excel("..\\PBData\\DATA3.xlsx")
hcdf = pd.read_csv("..\\HCData\\FullDataHC20191029\\Active_Mobiles_HC_Data.csv")

C:\Users\sgupta\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
print(pbdf.shape)
print(mesdq_df.shape)
print(hcdf.shape)

(3557518, 18)
(155791, 5)
(637826, 22)


In [57]:
pbdf["ACTIVITY_DATE"] = pd.to_datetime(pbdf['ACTIVITY_DATE'],format='%d%b%Y' )
mesdq_df['time_decision_date'] = pd.to_datetime(mesdq_df['time_decision_date'],format='%Y-%m-%d' )
pbdf['month'] = pbdf['ACTIVITY_DATE'].dt.strftime("%Y-%m")
hcdf['TIME_DECISION_DATE'] = pd.to_datetime(hcdf['TIME_DECISION_DATE'],format='%d-%m-%y' )

In [59]:
hcdf1 = pd.merge(hcdf,mesdq_df,how='left',left_on=['MOBILE','PRODUCT_GROUP','TIME_DECISION_DATE'],
                 right_on=['mobile','product_group','time_decision_date'])

In [60]:
hcdf1['mesdq'] = hcdf1['final_mes'] 
hcdf1.drop(columns=['mobile','member_id','product_group','time_decision_date','final_mes'],inplace=True )

In [61]:
df = pd.merge(pbdf,hcdf1,how="inner",left_on='DQ_MOBILE_PHONE',right_on = 'MOBILE')

In [62]:
df[pd.isna(df['mesdq_y']) ==False]['mesdq_x'] =  df[pd.isna(df['mesdq_y']) ==False]['mesdq_y']

C:\Users\sgupta\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [63]:
df["mesdq"] = df["mesdq_x"]
df.drop(columns=["mesdq_y",'mesdq_x'],inplace=True)
df['ACTIVITY_DATE'] = pd.to_datetime(df['ACTIVITY_DATE'],format='%d%b%Y' )
df= df[df['ACTIVITY_DATE']<df["TIME_DECISION_DATE"]]
df['weekend_flg'] = df['ACTIVITY_DATE'].apply(lambda x:1 if (x.day ==6)|(x.day ==7) else 0)


In [64]:
cust = df.groupby(["MOBILE","PRODUCT_GROUP"]).agg({"MEMBER_ID":['nunique'],
                        "ACTIVITY_ID": ['nunique'],
                        "ACTIVITY_VALUE" : ['max','min','median','std','mean'],
                        "ALLOCATED_POINTS": ['max','min','median','std','mean'],
                        "mesdq": ['max'],
                        "category":['nunique'],
                        "p_commercial_name":['nunique'],
                        "partner_type":['nunique'],
                        "TIME_DECISION_DATE" : ['max'],
                        "ACTIVITY_DATE" : ['min','max']
                         })
cust.columns = ["_".join(x) for x in cust.columns.ravel()]
cat=df["category"].unique()

In [65]:
for x in ["Earn","Burn"]:
    for y in cat:
        for z in [90,180,365,1000]:
            tmp = df[(df["type"] == x) & (df["category"]== y) & ((df["TIME_DECISION_DATE"]-df['ACTIVITY_DATE']).dt.days<=z)].groupby(['MOBILE','PRODUCT_GROUP']).agg({ "ACTIVITY_VALUE" : ['max','min','median','std','mean','sum'],
                        "ALLOCATED_POINTS": ['max','min','median','std','mean','sum'],
                        "ACTIVITY_ID": ['count'],
                        "category":['nunique'],
                        "month" : ['nunique'],
                        "ACTIVITY_DATE" : ['nunique']
                        })
            tmp.columns = [s + "_" + y + "_" + str(z) + "_"+ x for s in ["_".join(x) for x in tmp.columns.ravel()]]
            cust[[x for x in tmp.columns]] = tmp[[x for x in tmp.columns]]

for x in [x for x in cust.columns if '1000' in x]:
    cust[x.replace("1000","90_by_1000")] = cust[x.replace("1000","90")]/cust[x]
    cust[x.replace("1000","180_by_1000")] = cust[x.replace("1000","180")]/cust[x]
    cust[x.replace("1000","365_by_1000")] = cust[x.replace("1000","365")]/cust[x]

cust['MOB'] = (cust['TIME_DECISION_DATE_max']-cust["ACTIVITY_DATE_min"]).dt.days

for y in cat:
    for z in [90,180,365,1000]:
        tmp = df[(df["weekend_flg"] == 1) & (df["category"]== y) & ((df["TIME_DECISION_DATE"]-df['ACTIVITY_DATE']).dt.days<=z)].groupby(['MOBILE','PRODUCT_GROUP'])['ACTIVITY_ID'].count()
        cust["weekend_ratio_" + str(z) +"_"+y] = tmp

for x in ["Earn","Burn"]:
    for z in [90,180,365,1000]:
        tmp = df[(df["type"] == x) & ((df["TIME_DECISION_DATE"]-df['ACTIVITY_DATE']).dt.days<=z)].groupby(['MOBILE','PRODUCT_GROUP']).agg({ 
                    "ALLOCATED_POINTS": ['sum']
                    })
        tmp.columns = [s + "_" + str(z) + "_"+ x for s in ["_".join(x) for x in tmp.columns.ravel()]]
        cust[[x for x in tmp.columns]] = tmp[[x for x in tmp.columns]]
        
for x in [y for y in [x for x in cust.columns if 'Earn' in x ] if 'sum' in y]:
    cust[x.replace("Earn","Earn_by_Burn")] = cust[x]/cust[x.replace("Earn","Burn")]
    
df = df.sort_values(by =['MOBILE','PRODUCT_GROUP','ACTIVITY_DATE'],ascending=False)

cust["ALLOCATED_POINTS_EARN_LAG0"]=df[df["type"]=="Earn"].drop_duplicates(subset=['MOBILE','PRODUCT_GROUP'],keep='first').set_index(['MOBILE','PRODUCT_GROUP'])["ALLOCATED_POINTS"]
cust["ALLOCATED_POINTS_EARN_LAG1"] = df[df["type"]=="Earn"].set_index(['MOBILE','PRODUCT_GROUP']).groupby(['MOBILE','PRODUCT_GROUP']).shift(-1).reset_index().drop_duplicates(subset=['MOBILE','PRODUCT_GROUP']).set_index(['MOBILE','PRODUCT_GROUP'])["ALLOCATED_POINTS"]
cust["ALLOCATED_POINTS_EARN_LAG2"] = df[df["type"]=="Earn"].set_index(['MOBILE','PRODUCT_GROUP']).groupby(['MOBILE','PRODUCT_GROUP']).shift(-2).reset_index().drop_duplicates(subset=['MOBILE','PRODUCT_GROUP']).set_index(['MOBILE','PRODUCT_GROUP'])["ALLOCATED_POINTS"]
cust["ALLOCATED_POINTS_EARN_LAG3"] = df[df["type"]=="Earn"].set_index(['MOBILE','PRODUCT_GROUP']).groupby(['MOBILE','PRODUCT_GROUP']).shift(-3).reset_index().drop_duplicates(subset=['MOBILE','PRODUCT_GROUP']).set_index(['MOBILE','PRODUCT_GROUP'])["ALLOCATED_POINTS"]

cust["ALLOCATED_POINTS_BURN_LAG0"]=df[df["type"]=="Burn"].drop_duplicates(subset=['MOBILE','PRODUCT_GROUP'],keep='first').set_index(['MOBILE','PRODUCT_GROUP'])["ALLOCATED_POINTS"]
cust["ALLOCATED_POINTS_BURN_LAG1"] = df[df["type"]=="Burn"].set_index(['MOBILE','PRODUCT_GROUP']).groupby(['MOBILE','PRODUCT_GROUP']).shift(-1).reset_index().drop_duplicates(subset=['MOBILE','PRODUCT_GROUP']).set_index(['MOBILE','PRODUCT_GROUP'])["ALLOCATED_POINTS"]
cust["ALLOCATED_POINTS_BURN_LAG2"] = df[df["type"]=="Burn"].set_index(['MOBILE','PRODUCT_GROUP']).groupby(['MOBILE','PRODUCT_GROUP']).shift(-2).reset_index().drop_duplicates(subset=['MOBILE','PRODUCT_GROUP']).set_index(['MOBILE','PRODUCT_GROUP'])["ALLOCATED_POINTS"]
cust["ALLOCATED_POINTS_BURN_LAG3"] = df[df["type"]=="Burn"].set_index(['MOBILE','PRODUCT_GROUP']).groupby(['MOBILE','PRODUCT_GROUP']).shift(-3).reset_index().drop_duplicates(subset=['MOBILE','PRODUCT_GROUP']).set_index(['MOBILE','PRODUCT_GROUP'])["ALLOCATED_POINTS"]

In [66]:
cust.reset_index(inplace=True)

In [109]:
cust[cust["PRODUCT_GROUP"]=="CD"][CD_2YR_PREDICTORS].head()

,month_nunique_DEBIT_1000_Earn,mesdq_max,month_nunique_FUEL_1000_Earn,month_nunique_FUEL_365_by_1000_Earn,month_nunique_E_WALLET_1000_Earn,month_nunique_E_WALLET_180_by_1000_Burn,month_nunique_GROCERY_365_by_1000_Earn,month_nunique_FUEL_180_by_1000_Earn,month_nunique_GROCERY_1000_Earn,ALLOCATED_POINTS_mean_E_WALLET_180_by_1000_Burn,...,ALLOCATED_POINTS_sum_FUEL_365_by_1000_Earn,ACTIVITY_VALUE_max_FUEL_90_Earn,ACTIVITY_VALUE_std_E_WALLET_1000_Earn,ACTIVITY_VALUE_max_FUEL_180_Earn,ALLOCATED_POINTS_sum_GROCERY_180_by_1000_Earn,ALLOCATED_POINTS_max_FUEL_1000_Earn,ALLOCATED_POINTS_std_DEBIT_180_Earn,ALLOCATED_POINTS_sum_E_WALLET_180_Burn,ALLOCATED_POINTS_median_FUEL_180_Earn,ALLOCATED_POINTS_mean_FUEL_90_Earn
0,NaN,14.45,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,NaN,NaN
2,NaN,17.36,2.0,1.0,NaN,NaN,NaN,0.5,NaN,NaN,...,1.0,NaN,NaN,100.0,NaN,2.0,NaN,NaN,1.0,NaN
4,NaN,13.62,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,...,1.0,100.0,NaN,100.0,NaN,1.0,NaN,NaN,1.0,1.0
6,14.0,13.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.12132,NaN,NaN,NaN


In [67]:
cust.to_pickle("cust_mesdq_20191105.pkl")

In [110]:
cust["CD_2YR_OVERLAP_MDL_SCORE"] = -9999
cust["CL_2YR_OVERLAP_MDL_SCORE"] = -9999
cust["CD_FULL_OVERLAP_MDL_SCORE"] = -9999
cust["CL_FULL_OVERLAP_MDL_SCORE"] = -9999

In [111]:
loaded_modelcd2yr = pickle.load(open(r"..\\2yroverlap\\cd\\xgb_cd_fstq_wo_hc_mesdq",'rb'))

In [112]:
CD_2YR_PREDICTORS = ['month_nunique_DEBIT_1000_Earn',
  'mesdq_max',
  'month_nunique_FUEL_1000_Earn',
  'month_nunique_FUEL_365_by_1000_Earn',
  'month_nunique_E_WALLET_1000_Earn',
  'month_nunique_E_WALLET_180_by_1000_Burn',
  'month_nunique_GROCERY_365_by_1000_Earn',
  'month_nunique_FUEL_180_by_1000_Earn',
  'month_nunique_GROCERY_1000_Earn',
  'ALLOCATED_POINTS_mean_E_WALLET_180_by_1000_Burn',
  'ALLOCATED_POINTS_median_E_WALLET_180_by_1000_Burn',
  'ACTIVITY_VALUE_sum_FUEL_365_by_1000_Earn',
  'ALLOCATED_POINTS_max_FUEL_180_Earn',
  'ALLOCATED_POINTS_sum_E_WALLET_180_by_1000_Earn',
  'ALLOCATED_POINTS_max_FUEL_90_Earn',
  'month_nunique_E_WALLET_180_by_1000_Earn',
  'ALLOCATED_POINTS_std',
  'ALLOCATED_POINTS_sum_E_WALLET_180_by_1000_Burn',
  'MOB',
  'ACTIVITY_VALUE_sum_GROCERY_90_by_1000_Earn',
  'month_nunique_FUEL_365_Earn',
  'ALLOCATED_POINTS_mean_FUEL_180_Earn',
  'month_nunique_DEBIT_365_by_1000_Earn',
  'ALLOCATED_POINTS_median_FUEL_365_Earn',
  'ACTIVITY_VALUE_max_GROCERY_90_by_1000_Earn',
  'ALLOCATED_POINTS_std_DEBIT_365_Earn',
  'ACTIVITY_ID_count_FUEL_365_by_1000_Earn',
  'ACTIVITY_VALUE_std',
  'ALLOCATED_POINTS_min_E_WALLET_365_Burn',
  'ALLOCATED_POINTS_max_DEBIT_365_Earn',
  'ALLOCATED_POINTS_sum_FUEL_365_by_1000_Earn',
  'ACTIVITY_VALUE_max_FUEL_90_Earn',
  'ACTIVITY_VALUE_std_E_WALLET_1000_Earn',
  'ACTIVITY_VALUE_max_FUEL_180_Earn',
  'ALLOCATED_POINTS_sum_GROCERY_180_by_1000_Earn',
  'ALLOCATED_POINTS_max_FUEL_1000_Earn',
  'ALLOCATED_POINTS_std_DEBIT_180_Earn',
  'ALLOCATED_POINTS_sum_E_WALLET_180_Burn',
  'ALLOCATED_POINTS_median_FUEL_180_Earn',
  'ALLOCATED_POINTS_mean_FUEL_90_Earn']

In [130]:
cust.loc[cust["PRODUCT_GROUP"]=="CD","CD_2YR_OVERLAP_MDL_SCORE"] = loaded_modelcd2yr.predict_proba(
    cust[cust["PRODUCT_GROUP"]=="CD"][CD_2YR_PREDICTORS],ntree_limit= loaded_modelcd2yr.best_ntree_limit)[:,1]

In [96]:
loaded_modelcl2yr = pickle.load(open(r"..\\2yroverlap\\clx\\xgb_cd_fstq_wo_hc_cl_mix",'rb'))

In [97]:
CL_2YR_PREDICTORS = ['ACTIVITY_DATE_nunique_E_WALLET_180_Burn',
  'mesdq_max',
  'ALLOCATED_POINTS_mean_E_WALLET_180_by_1000_Burn',
  'month_nunique_E_WALLET_180_by_1000_Burn',
  'ACTIVITY_VALUE_mean_FUEL_1000_Earn',
  'ACTIVITY_ID_count_E_WALLET_180_by_1000_Burn',
  'MOB',
  'month_nunique_E_WALLET_180_Burn',
  'ACTIVITY_VALUE_sum_FUEL_365_by_1000_Earn',
  'ALLOCATED_POINTS_sum_E_WALLET_180_by_1000_Burn',
  'month_nunique_GROCERY_1000_Earn',
  'ACTIVITY_VALUE_median_FUEL_1000_Earn',
  'ALLOCATED_POINTS_min_E_WALLET_365_Burn',
  'ALLOCATED_POINTS_sum_FUEL_1000_Earn',
  'month_nunique_E_WALLET_180_by_1000_Earn',
  'ACTIVITY_DATE_nunique_DEBIT_365_Earn',
  'month_nunique_DEBIT_1000_Earn',
  'ALLOCATED_POINTS_mean_FUEL_365_by_1000_Earn',
  'ACTIVITY_VALUE_sum_GROCERY_180_by_1000_Earn',
  'ALLOCATED_POINTS_sum_FUEL_365_by_1000_Earn',
  'ALLOCATED_POINTS_min_E_WALLET_180_Burn',
  'ACTIVITY_ID_count_FUEL_365_by_1000_Earn',
  'ALLOCATED_POINTS_EARN_LAG3',
  'ALLOCATED_POINTS_max_DEBIT_365_Earn',
  'ALLOCATED_POINTS_median_FUEL_1000_Earn',
  'ACTIVITY_VALUE_std',
  'ALLOCATED_POINTS_sum_GROCERY_90_by_1000_Earn',
  'ALLOCATED_POINTS_sum_E_WALLET_180_Burn',
  'ACTIVITY_VALUE_median_E_WALLET_1000_Earn',
  'ALLOCATED_POINTS_max_APPAREL_365_Earn',
  'ALLOCATED_POINTS_sum_GROCERY_180_by_1000_Earn',
  'ACTIVITY_VALUE_std_E_WALLET_1000_Earn',
  'ACTIVITY_ID_count_GROCERY_180_by_1000_Earn',
  'ALLOCATED_POINTS_max_DEBIT_1000_Earn',
  'ALLOCATED_POINTS_std_DEBIT_1000_Earn',
  'ACTIVITY_VALUE_median',
  'ALLOCATED_POINTS_sum_DEBIT_1000_Earn',
  'ALLOCATED_POINTS_EARN_LAG1',
  'ALLOCATED_POINTS_sum_365_Earn_by_Burn',
  'ALLOCATED_POINTS_EARN_LAG2',
  'ALLOCATED_POINTS_sum_1000_Earn_by_Burn',
  'ALLOCATED_POINTS_sum_GROCERY_365_by_1000_Earn']

In [137]:
cust.loc[cust["PRODUCT_GROUP"]=="CL","CL_2YR_OVERLAP_MDL_SCORE"] = loaded_modelcl2yr.predict_proba(
    cust[cust["PRODUCT_GROUP"]=="CL"][CL_2YR_PREDICTORS],
     ntree_limit= loaded_modelcl2yr.best_ntree_limit)[:,1]

In [138]:
loaded_modelcd_full = pickle.load(open(r"..\\FullDataAnalysis\\xgb_cd_fstq_wo_hc1.mod",'rb'))

CD_FULL_PREDICTORS = ['ALLOCATED_POINTS_std',
  'ALLOCATED_POINTS_EARN_LAG2',
  'ACTIVITY_ID_nunique',
  'ALLOCATED_POINTS_EARN_LAG1',
  'mesdq_max',
  'month_nunique_FUEL_1000_Earn',
  'ALLOCATED_POINTS_EARN_LAG3',
  'month_nunique_DEBIT_1000_Earn',
  'ACTIVITY_VALUE_std',
  'month_nunique_DEBIT_180_Earn',
  'month_nunique_GROCERY_1000_Earn',
  'month_nunique_DEBIT_365_Earn',
  'month_nunique_DEBIT_90_Earn',
  'month_nunique_E_WALLET_365_by_1000_Earn',
  'ACTIVITY_ID_count_DEBIT_180_Earn',
  'month_nunique_FUEL_365_Earn',
  'month_nunique_FUEL_180_by_1000_Earn',
  'month_nunique_DEBIT_90_by_1000_Earn',
  'ALLOCATED_POINTS_sum_E_WALLET_365_by_1000_Earn',
  'ACTIVITY_ID_count_FUEL_1000_Earn',
  'MOB',
  'ACTIVITY_ID_count_FUEL_365_by_1000_Earn',
  'ALLOCATED_POINTS_max_FUEL_180_Earn',
  'ACTIVITY_DATE_nunique_E_WALLET_365_Earn',
  'ACTIVITY_DATE_nunique_DEBIT_180_by_1000_Earn',
  'ALLOCATED_POINTS_std_DEBIT_90_Earn',
  'ALLOCATED_POINTS_median_DEBIT_90_Earn',
  'ACTIVITY_DATE_nunique_DEBIT_365_by_1000_Earn',
  'ACTIVITY_VALUE_sum_E_WALLET_180_by_1000_Earn',
  'month_nunique_E_WALLET_1000_Burn',
  'ALLOCATED_POINTS_max_FUEL_365_Earn',
  'ALLOCATED_POINTS_mean_FUEL_180_Earn',
  'ACTIVITY_VALUE_sum_FUEL_1000_Earn',
  'ACTIVITY_VALUE_sum_E_WALLET_365_by_1000_Earn']

cust.loc[cust["PRODUCT_GROUP"]=="CD","CD_FULL_OVERLAP_MDL_SCORE"] = loaded_modelcd_full.predict_proba(
    cust[cust["PRODUCT_GROUP"]=="CD"][CD_FULL_PREDICTORS],
     ntree_limit= loaded_modelcd_full.best_ntree_limit)[:,1]

In [139]:
loaded_modelcl_full = pickle.load(open(r"..\\FullDataAnalysis\\xgb_cl_fstq_wo_hc.mod",'rb'))

CL_FULL_PREDICTORS = ['ACTIVITY_VALUE_std',
  'month_nunique_DEBIT_1000_Earn',
  'mesdq_max',
  'month_nunique_GROCERY_1000_Earn',
  'ALLOCATED_POINTS_std',
  'month_nunique_DEBIT_90_by_1000_Earn',
  'ALLOCATED_POINTS_std_GROCERY_1000_Earn',
  'ALLOCATED_POINTS_sum_DEBIT_90_by_1000_Earn',
  'ALLOCATED_POINTS_mean_E_WALLET_365_Earn',
  'ALLOCATED_POINTS_EARN_LAG3',
  'ALLOCATED_POINTS_std_E_WALLET_1000_Earn',
  'ALLOCATED_POINTS_median_E_WALLET_180_Earn',
  'month_nunique_FUEL_1000_Earn',
  'ALLOCATED_POINTS_mean_DEBIT_90_by_1000_Earn',
  'ALLOCATED_POINTS_max_APPAREL_365_Earn',
  'ALLOCATED_POINTS_sum_365_Burn',
  'ALLOCATED_POINTS_mean_E_WALLET_180_Earn',
  'ALLOCATED_POINTS_max_DEBIT_1000_Earn',
  'ALLOCATED_POINTS_mean_E_WALLET_365_by_1000_Earn',
  'ALLOCATED_POINTS_sum_DEBIT_1000_Earn',
  'ACTIVITY_VALUE_median_GROCERY_1000_Earn',
  'ALLOCATED_POINTS_sum_180_Burn',
  'ACTIVITY_VALUE_median_E_WALLET_180_Earn']

cust.loc[cust["PRODUCT_GROUP"]=="CL","CL_FULL_OVERLAP_MDL_SCORE"] = loaded_modelcl_full.predict_proba(
    cust[cust["PRODUCT_GROUP"]=="CL"][CL_FULL_PREDICTORS],
     ntree_limit= loaded_modelcl_full.best_ntree_limit)[:,1]

In [140]:
[x for x in cust.columns if "OVERLAP" in x]

['CD_2YR_OVERLAP_MDL_SCORE',
 'CL_2YR_OVERLAP_MDL_SCORE',
 'CD_FULL_OVERLAP_MDL_SCORE',
 'CL_FULL_OVERLAP_MDL_SCORE']

In [141]:
relevant_cols  = ['MOBILE',"PRODUCT_GROUP","CD_2YR_OVERLAP_MDL_SCORE",'CL_2YR_OVERLAP_MDL_SCORE',
 'CD_FULL_OVERLAP_MDL_SCORE',
 'CL_FULL_OVERLAP_MDL_SCORE',]

In [144]:
final = pd.merge(hcdf,cust[relevant_cols],on=["MOBILE","PRODUCT_GROUP"],how = "left")

In [145]:
final[(final["CD_2YR_OVERLAP_MDL_SCORE"] != -9999) & (final["PRODUCT_GROUP"] == "CD") ]

,PRODUCT_GROUP,MOBILE,HC_SCORE,CLIENT_EXI_3M,SKP_CREDIT_CASE,TIME_DECISION_DATE,SCORE_CB,SCORE_CB_BAND,CLIENT_GENDER,CLIENT_AGE_YEARS,...,RISK_FSPD30,RISK_AGRF60,RISK_FSTPD30,RISK_AGRF90,RISK_FSTQPD30,RISK_AGRF120,CD_2YR_OVERLAP_MDL_SCORE,CL_2YR_OVERLAP_MDL_SCORE,CD_FULL_OVERLAP_MDL_SCORE,CL_FULL_OVERLAP_MDL_SCORE
104597,CD,9994294070,0.010803,N,227810922,2019-03-07,1.0,b_1_5,M,32.54,...,0,1,0,1,0,1,0.314936,-9999.0,0.525112,-9999.0
104598,CD,7433082354,0.013201,Y,233115690,2019-04-19,726.0,e_680+,M,22.52,...,0,1,0,1,0,1,0.352182,-9999.0,0.454600,-9999.0
104599,CD,8446667663,NaN,NaN,77353612,2015-08-05,-1.0,a_-1,M,24.02,...,0,1,0,1,0,1,NaN,NaN,NaN,NaN
104600,CD,9654752572,0.020422,Y,171928958,2018-02-24,671.0,d_566_680,M,26.03,...,0,1,0,1,0,1,NaN,NaN,NaN,NaN
104601,CD,9814330190,0.033336,Y,213950051,2018-11-07,722.0,e_680+,M,28.64,...,0,1,0,1,0,1,0.306889,-9999.0,0.466258,-9999.0
104602,CD,9836328609,0.015972,N,206384251,2018-09-03,-1.0,a_-1,M,25.38,...,0,1,0,1,0,1,0.348055,-9999.0,0.501652,-9999.0
104603,CD,9991955500,0.003738,N,206087380,2018-08-31,-1.0,a_-1,M,34.15,...,0,1,0,1,0,1,0.229889,-9999.0,0.432529,-9999.0
104605,CD,9611080238,0.004767,N,198284553,2018-07-01,663.0,d_566_680,M,43.52,...,0,1,1,1,1,1,0.177984,-9999.0,0.395322,-9999.0
104606,CD,9592229584,0.033048,Y,207034894,2018-09-09,-1.0,a_-1,M,28.69,...,0,1,0,1,0,1,0.465405,-9999.0,0.532557,-9999.0
104607,CD,9030982928,0.019307,N,135722241,2017-10-13,727.0,e_680+,M,41.34,...,0,1,0,1,0,1,0.301385,-9999.0,0.433805,-9999.0


In [146]:
final1 = final[pd.isna(final["HC_SCORE"])==False]

In [ ]:
final

In [153]:
y_pred = final1[(final1["PRODUCT_GROUP"]=="CD") & (pd.isna(final1["CD_2YR_OVERLAP_MDL_SCORE"]) == False)]["CD_2YR_OVERLAP_MDL_SCORE"]
y      = final1[(final1["PRODUCT_GROUP"]=="CD") & (pd.isna(final1["CD_2YR_OVERLAP_MDL_SCORE"]) == False)]["RISK_FSTQPD30"]
print ("Gini for CD_2YR_OVERLAP_MDL_SCORE ",metrics.roc_auc_score(y,y_pred)*2-1)

Gini for CD_2YR_OVERLAP_MDL_SCORE  0.13241069846937914


In [154]:
y_pred = final1[(final1["PRODUCT_GROUP"]=="CL") & (pd.isna(final1["CL_2YR_OVERLAP_MDL_SCORE"]) == False)]["CL_2YR_OVERLAP_MDL_SCORE"]
y      = final1[(final1["PRODUCT_GROUP"]=="CL") & (pd.isna(final1["CL_2YR_OVERLAP_MDL_SCORE"]) == False)]["RISK_FSTQPD30"]
print ("Gini for CL_2YR_OVERLAP_MDL_SCORE ",metrics.roc_auc_score(y,y_pred)*2-1)

Gini for CL_2YR_OVERLAP_MDL_SCORE  0.1322295548593173


In [156]:
y_pred = final1[(final1["PRODUCT_GROUP"]=="CD") & (pd.isna(final1["CD_FULL_OVERLAP_MDL_SCORE"]) == False)]["CD_FULL_OVERLAP_MDL_SCORE"]
y      = final1[(final1["PRODUCT_GROUP"]=="CD") & (pd.isna(final1["CD_FULL_OVERLAP_MDL_SCORE"]) == False)]["RISK_FSTQPD30"]
print ("Gini for CD_FULL_OVERLAP_MDL_SCORE ",metrics.roc_auc_score(y,y_pred)*2-1)

Gini for CD_FULL_OVERLAP_MDL_SCORE  0.1666524602738919


In [157]:
y_pred = final1[(final1["PRODUCT_GROUP"]=="CL") & (pd.isna(final1["CL_FULL_OVERLAP_MDL_SCORE"]) == False)]["CL_FULL_OVERLAP_MDL_SCORE"]
y      = final1[(final1["PRODUCT_GROUP"]=="CL") & (pd.isna(final1["CL_FULL_OVERLAP_MDL_SCORE"]) == False)]["RISK_FSTQPD30"]
print ("Gini for CL_FULL_OVERLAP_MDL_SCORE ",metrics.roc_auc_score(y,y_pred)*2-1)

Gini for CL_FULL_OVERLAP_MDL_SCORE  0.24369497420373043


In [159]:
final1.to_csv("final_20191107_scored.csv")